In [48]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler

In [49]:
df = pd.read_csv("fibonacci_sequence.csv")
print(df.head())

   Iteration Value
0          0     0
1          1     1
2          2     1
3          3     2
4          4     3


In [50]:
# Tomemos solo primeros 50 términos para evitar overflow
values = df["Value"].astype(float).values[:50]

# Normalizamos desde el inicio
values = values / np.max(values)

In [51]:
X, y = [], []
window_size = 5 #how many you want to see in the backwards
for i in range(len(values) - window_size):
    X.append(values[i:i+window_size])
    y.append(values[i+window_size])
    

X = np.array(X).reshape(-1, window_size, 1)
y = np.array(y).reshape(-1, 1)


In [52]:
print(X[:3])
print(y[:3])

[[[0.00000000e+00]
  [1.28555491e-10]
  [1.28555491e-10]
  [2.57110981e-10]
  [3.85666472e-10]]

 [[1.28555491e-10]
  [1.28555491e-10]
  [2.57110981e-10]
  [3.85666472e-10]
  [6.42777453e-10]]

 [[1.28555491e-10]
  [2.57110981e-10]
  [3.85666472e-10]
  [6.42777453e-10]
  [1.02844392e-09]]]
[[6.42777453e-10]
 [1.02844392e-09]
 [1.67122138e-09]]


In [53]:
#You need to make now reshape the keras, so it has (labels, window_size, characteristics)

"""
X
n_muestras → cuántas secuencias tienes (por ejemplo, 50 ventanas distintas).

pasos_de_tiempo → cuántos elementos hay en cada secuencia (en tu caso, 5 valores Fibonacci por ventana).

características → cuántos valores distintos hay en cada paso (aquí solo 1, el valor mismo).
"""
X = X.astype(np.float32)
y = y.astype(np.float32)

In [54]:
optimizer = keras.optimizers.AdamW(learning_rate=1e-5, clipnorm=1.0)

model = keras.Sequential([
    keras.layers.SimpleRNN(50, activation="tanh", input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(1)
])

#Mean Squared Error (y - yhat)^2
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [55]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_5 (SimpleRNN)        │ (None, 50)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,651 (10.36 KB)

 Trainable params: 2,651 (10.36 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=50)

model.compile()

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0166 - mae: 0.0353 - val_loss: 0.0137 - val_mae: 0.0450
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0165 - mae: 0.0352 - val_loss: 0.0137 - val_mae: 0.0450
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0165 - mae: 0.0353 - val_loss: 0.0136 - val_mae: 0.0450
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0164 - mae: 0.0353 - val_loss: 0.0136 - val_mae: 0.0450
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0164 - mae: 0.0353 - val_loss: 0.0135 - val_mae: 0.0451
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0163 - mae: 0.0354 - val_loss: 0.0135 - val_mae: 0.0451
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0163 - mae: 0.0354 - val_loss: 0.0134 - val_mae: 0.0451
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0163 - mae: 0.0355 - val_loss: 0.0134 - val_mae: 0.0451
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.0162 - mae: 0.0356 -

In [57]:
loss, mae = model.evaluate(X_test, Y_test)
print(f"Loss (Mean Squared Error): {loss}")
print(f"Mean Absolute Error: {mae}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - loss: 0.0118 - mae: 0.0469
Loss (Mean Squared Error): 0.011803540401160717
Mean Absolute Error: 0.04687332361936569


In [58]:
y_pred = model.predict(X_test)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
[[0.0433643 ]
 [0.00944708]
 [0.00947199]
 [0.23962177]
 [0.01436252]
 [0.09817878]
 [0.00940679]
 [0.00940687]
 [0.0094068 ]
 [0.00940679]
 [0.00940679]
 [0.00943169]
 [0.01057669]
 [0.00940903]]


In [59]:
import matplotlib.pyplot as plt

plt.plot(Y_test_original, label='Real')
plt.plot(y_pred_original, label='Predicción')
plt.legend()
plt.show()

NameError: name 'Y_test_original' is not defined